In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
from datetime import datetime, timedelta
import os
import joblib

# File paths
ACCOUNT_PATH = "/content/drive/MyDrive/fraud_detection/accounts.csv"
APPROVED_LOG = "/content/drive/MyDrive/fraud_detection/approved_transactions.csv"
FRAUD_LOG = "/content/drive/MyDrive/fraud_detection/fraud_transactions.csv"
MODEL_PATH = "/content/drive/MyDrive/fraud_detection/fraud_detection_model.pkl"

# Load model
model = joblib.load(MODEL_PATH)

# Microtransaction threshold
MICRO_AMOUNT_THRESHOLD = 5
MICRO_TX_LIMIT = 5  # block on 6th micro txn

def recent_micro_transactions(sender):
    """Returns count of small transactions in last 1 hour by this sender"""
    if not os.path.exists(APPROVED_LOG):
        return 0

    df = pd.read_csv(APPROVED_LOG)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    one_hour_ago = datetime.now() - timedelta(hours=1)
    recent = df[
        (df['From'] == sender) &
        (df['Amount'] <= MICRO_AMOUNT_THRESHOLD) &
        (df['Timestamp'] >= one_hour_ago)
    ]
    return len(recent)

# Main transaction handler
def handle_transaction(sender, receiver, amount, third_party=False):
    df = pd.read_csv(ACCOUNT_PATH)

    if sender == receiver:
        return "Sender and Receiver cannot be the same.", "", "", "", ""

    try:
        amount = float(amount)
    except:
        return "Invalid amount.", "", "", "", ""

    if amount <= 0:
        return "Invalid amount.", "", "", "", ""

    sender_balance_old = df.loc[df['Account Number'] == sender, 'Balance'].values[0]
    receiver_balance_old = df.loc[df['Account Number'] == receiver, 'Balance'].values[0]

    if sender_balance_old < amount:
        return "❌ Insufficient balance.", f"₹{sender_balance_old:.2f}", "", "", ""

    # Microtransaction fraud check
    micro_count = recent_micro_transactions(sender)
    if amount <= MICRO_AMOUNT_THRESHOLD and micro_count >= MICRO_TX_LIMIT:
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Attempted Amount": amount,
            "Status": "Fraud Blocked (Microtransaction Frequency)",
            "Sender Balance": sender_balance_old
        }]).to_csv(FRAUD_LOG, mode='a', index=False, header=not os.path.exists(FRAUD_LOG))

        return "🚨 Fraud Blocked (Microtransaction Frequency)", f"₹{sender_balance_old:.2f}", "", "", ""

    # AI-based detection
    features = [[amount, sender_balance_old, receiver_balance_old, int(third_party)]]
    prediction = model.predict(features)[0]
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if prediction == 0:
        # Approve
        df.loc[df['Account Number'] == sender, 'Balance'] -= amount
        df.loc[df['Account Number'] == receiver, 'Balance'] += amount
        df.to_csv(ACCOUNT_PATH, index=False)

        sender_balance_new = sender_balance_old - amount
        receiver_balance_new = receiver_balance_old + amount

        pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Amount": amount,
            "Status": "Success",
            "Sender Balance After": sender_balance_new,
            "Receiver Balance After": receiver_balance_new
        }]).to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return (
            "✅ Transaction Approved (AI Verified)",
            f"Sender Old: ₹{sender_balance_old:.2f}",
            f"Sender New: ₹{sender_balance_new:.2f}",
            f"Receiver Old: ₹{receiver_balance_old:.2f}",
            f"Receiver New: ₹{receiver_balance_new:.2f}"
        )

    else:
        # Blocked by AI
        pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Attempted Amount": amount,
            "Status": "Fraud Blocked (AI)",
            "Sender Balance": sender_balance_old
        }]).to_csv(FRAUD_LOG, mode='a', index=False, header=not os.path.exists(FRAUD_LOG))

        return (
            "🚨 Fraud Detected by AI! Transaction Blocked.",
            f"₹{sender_balance_old:.2f}",
            "",
            f"₹{receiver_balance_old:.2f}",
            ""
        )


In [3]:
import pandas as pd
import numpy as np

# Load the existing dataset
df = pd.read_csv("/content/drive/MyDrive/fraud_detection/fraud_dataset.csv")

# Check original columns
print("Original Columns:", df.columns.tolist())

# Add 'third_party_involved' = 0 to existing rows
df['third_party_involved'] = 0

# Simulate third-party interference
num_third_party_cases = 300
third_party_data = []

for _ in range(num_third_party_cases):
    sender = np.random.choice(df['from_acc'])
    receiver = np.random.choice(df['to_acc'])
    amount = np.random.uniform(500, 10000)
    balance_sender = np.random.uniform(amount, amount + 5000)
    balance_receiver = np.random.uniform(1000, 10000)

    # Make sure columns match the existing structure + new column
    third_party_data.append({
        'sender': sender,
        'receiver': receiver,
        'amount': amount,
        'sender_balance': balance_sender,
        'receiver_balance': balance_receiver,
        'is_fraud': 1,
        'third_party_involved': 1
    })

# Create DataFrame from simulated third-party fraud data
third_party_df = pd.DataFrame(third_party_data)

# Combine with original data
combined_df = pd.concat([df, third_party_df], ignore_index=True)

# Save the updated dataset
combined_df.to_csv("/content/drive/MyDrive/fraud_detection/fraud_dataset_updated.csv", index=False)
print("✅ Updated dataset with third-party fraud simulation saved.")


Original Columns: ['from_acc', 'to_acc', 'amount', 'sender_balance', 'receiver_balance', 'time_of_day', 'is_fraud']
✅ Updated dataset with third-party fraud simulation saved.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load updated dataset
df = pd.read_csv("/content/drive/MyDrive/fraud_detection/fraud_dataset_updated.csv")

# Features and label
X = df[['amount', 'sender_balance', 'receiver_balance', 'third_party_involved']]
y = df['is_fraud']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "/content/drive/MyDrive/fraud_detection/fraud_detection_model.pkl")

print("✅ AI Model trained and saved successfully!")


✅ AI Model trained and saved successfully!


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import joblib
import os

# Paths
ACCOUNT_PATH = "/content/drive/MyDrive/fraud_detection/accounts.csv"
APPROVED_LOG = "/content/drive/MyDrive/fraud_detection/approved_transactions.csv"
FRAUD_LOG = "/content/drive/MyDrive/fraud_detection/fraud_transactions.csv"
MODEL_PATH = "/content/drive/MyDrive/fraud_detection/fraud_detection_model.pkl"

# Load AI Model
model = joblib.load(MODEL_PATH)

# Core transaction function
def handle_transaction(sender, receiver, amount, third_party=False):
    df = pd.read_csv(ACCOUNT_PATH)

    if sender == receiver:
        return "❌ Sender and Receiver cannot be the same.", "", "", "", ""

    try:
        amount = float(amount)
        if amount <= 0:
            return "❌ Invalid amount.", "", "", "", ""
    except:
        return "❌ Invalid amount.", "", "", "", ""

    sender_balance_old = df.loc[df['Account Number'] == sender, 'Balance'].values[0]
    receiver_balance_old = df.loc[df['Account Number'] == receiver, 'Balance'].values[0]

    now = datetime.now()
    now_str = now.strftime("%Y-%m-%d %H:%M:%S")

    # ---------------- RULE-BASED MICROTRANSACTION CHECK ---------------- #
    suspicious_micro = False
    if os.path.exists(APPROVED_LOG):
        history = pd.read_csv(APPROVED_LOG)
        history['Timestamp'] = pd.to_datetime(history['Timestamp'], errors='coerce')
        past_hour = now - timedelta(hours=1)

        recent_micro = history[
            (history['From'] == sender) &
            (history['Amount'] <= 5) &  # low value
            (history['Timestamp'] >= past_hour)
        ]

        if len(recent_micro) >= 5:
            suspicious_micro = True

    # ---------------- AI PREDICTION ---------------- #
    third_party_flag = 1 if third_party else 0
    features = [[amount, sender_balance_old, receiver_balance_old, third_party_flag]]
    ai_prediction = model.predict(features)[0]

    # ---------------- FINAL DECISION ---------------- #
    if ai_prediction == 0 and not suspicious_micro and sender_balance_old >= amount:
        # Approve
        df.loc[df['Account Number'] == sender, 'Balance'] -= amount
        df.loc[df['Account Number'] == receiver, 'Balance'] += amount
        df.to_csv(ACCOUNT_PATH, index=False)

        sender_new = sender_balance_old - amount
        receiver_new = receiver_balance_old + amount

        approved = pd.DataFrame([{
            "Timestamp": now_str,
            "From": sender,
            "To": receiver,
            "Amount": amount,
            "Status": "Success",
            "Sender Balance After": sender_new,
            "Receiver Balance After": receiver_new
        }])
        approved.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return (
            "✅ Transaction Successful (AI + Rules Verified)",
            f"Sender Old: ₹{sender_balance_old:.2f}",
            f"Sender New: ₹{sender_new:.2f}",
            f"Receiver Old: ₹{receiver_balance_old:.2f}",
            f"Receiver New: ₹{receiver_new:.2f}"
        )
    else:
        # Block fraud
        reason = []
        if ai_prediction == 1:
            reason.append("AI Model")
        if suspicious_micro:
            reason.append("Microtransaction Frequency")
        if sender_balance_old < amount:
            reason.append("Insufficient Balance")

        fraud = pd.DataFrame([{
            "Timestamp": now_str,
            "From": sender,
            "To": receiver,
            "Attempted Amount": amount,
            "Status": f"Blocked: {' + '.join(reason)}",
            "Sender Balance": sender_balance_old,
            "Third Party": third_party
        }])
        fraud.to_csv(FRAUD_LOG, mode='a', index=False, header=not os.path.exists(FRAUD_LOG))

        return (
            f"🚨 Fraud Blocked ({' + '.join(reason)})",
            f"Sender Balance: ₹{sender_balance_old:.2f}",
            "",
            f"Receiver Balance: ₹{receiver_balance_old:.2f}",
            ""
        )


In [6]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00


In [27]:
import gradio as gr
import pandas as pd
from datetime import datetime
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# File paths
ACCOUNT_PATH = "/content/drive/MyDrive/fraud_detection/accounts.csv"
APPROVED_LOG = "/content/drive/MyDrive/fraud_detection/approved_transactions.csv"
FRAUD_LOG = "/content/drive/MyDrive/fraud_detection/fraud_transactions.csv"
MODEL_PATH = "/content/drive/MyDrive/fraud_detection/fraud_model.pkl"

# Load model
model = joblib.load(MODEL_PATH)

# Load accounts
def get_accounts():
    return sorted(pd.read_csv(ACCOUNT_PATH)['Account Number'].tolist())

# Transaction Handler
def handle_transaction(sender, receiver, amount):
    df = pd.read_csv(ACCOUNT_PATH)

    if sender == receiver:
        return "Sender and Receiver cannot be the same.", "", "", "", ""

    try:
        amount = float(amount)
    except:
        return "Invalid amount.", "", "", "", ""

    if amount <= 0:
        return "Invalid amount.", "", "", "", ""

    try:
        sender_balance_old = df.loc[df['Account Number'] == sender, 'Balance'].values[0]
        receiver_balance_old = df.loc[df['Account Number'] == receiver, 'Balance'].values[0]
    except IndexError:
        return "Sender or Receiver account not found.", "", "", "", ""

    if sender_balance_old < amount:
        return "❌ Insufficient balance in sender's account.", f"₹{sender_balance_old:.2f}", "", "", ""

    # AI Prediction
    features = pd.DataFrame([[amount, sender_balance_old, receiver_balance_old]], columns=['amount', 'sender_balance', 'receiver_balance'])
    prediction = model.predict(features)[0]

    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if prediction == 0:
        df.loc[df['Account Number'] == sender, 'Balance'] -= amount
        df.loc[df['Account Number'] == receiver, 'Balance'] += amount
        df.to_csv(ACCOUNT_PATH, index=False)

        sender_balance_new = sender_balance_old - amount
        receiver_balance_new = receiver_balance_old + amount

        approved = pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Amount": amount,
            "Status": "Success",
            "Sender Balance After": sender_balance_new,
            "Receiver Balance After": receiver_balance_new
        }])
        approved.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return (
            "✅ Transaction Successful (AI Verified)",
            f"Sender Old: ₹{sender_balance_old:.2f}",
            f"Sender New: ₹{sender_balance_new:.2f}",
            f"Receiver Old: ₹{receiver_balance_old:.2f}",
            f"Receiver New: ₹{receiver_balance_new:.2f}"
        )
    else:
        fraud = pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Attempted Amount": amount,
            "Status": "Fraud Blocked (AI)",
            "Sender Balance": sender_balance_old
        }])
        fraud.to_csv(FRAUD_LOG, mode='a', index=False, header=not os.path.exists(FRAUD_LOG))

        return (
            "🚨 Fraud Detected by AI! Transaction Blocked.",
            f"₹{sender_balance_old:.2f}",
            "",
            f"₹{receiver_balance_old:.2f}",
            ""
        )

# Deposit
def deposit(account, amount):
    try:
        df = pd.read_csv(ACCOUNT_PATH)
        amount = float(amount)
        if amount <= 0:
            return "Invalid deposit amount."

        old_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        df.loc[df['Account Number'] == account, 'Balance'] += amount
        new_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        df.to_csv(ACCOUNT_PATH, index=False)

        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log = pd.DataFrame([{
            "Timestamp": now,
            "From": "CASH",
            "To": account,
            "Amount": amount,
            "Status": "Deposit",
            "Sender Balance After": "-",
            "Receiver Balance After": new_balance
        }])
        log.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return f"✅ ₹{amount:.2f} deposited successfully. Old: ₹{old_balance:.2f}, New: ₹{new_balance:.2f}"
    except Exception as e:
        return f"❌ Deposit failed: {str(e)}"

# Withdraw
def withdraw(account, amount):
    try:
        df = pd.read_csv(ACCOUNT_PATH)
        amount = float(amount)
        current_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        if amount <= 0:
            return "Invalid withdraw amount."
        if amount > current_balance:
            return "Insufficient balance."

        old_balance = current_balance
        df.loc[df['Account Number'] == account, 'Balance'] -= amount
        new_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        df.to_csv(ACCOUNT_PATH, index=False)

        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log = pd.DataFrame([{
            "Timestamp": now,
            "From": account,
            "To": "CASH",
            "Amount": amount,
            "Status": "Withdraw",
            "Sender Balance After": new_balance,
            "Receiver Balance After": "-"
        }])
        log.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return f"✅ ₹{amount:.2f} withdrawn successfully. Old: ₹{old_balance:.2f}, New: ₹{new_balance:.2f}"
    except Exception as e:
        return f"❌ Withdrawal failed: {str(e)}"

# Balance View
def get_balance(account):
    df = pd.read_csv(ACCOUNT_PATH)
    balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
    return f"Current Balance: ₹{balance:.2f}"

  # Get Account Holder Info
def get_account_info(account):
    try:
        df = pd.read_csv(ACCOUNT_PATH)
        user = df[df['Account Number'] == account].iloc[0]
        info = f"""
Name: {user['Name']}
Phone: {user['Phone']}
Address: {user['Address']}
Balance: ₹{user['Balance']:.2f}
        """
        return info.strip()
    except Exception as e:
        return f"Error: {e}"


# Transaction History
def view_history(account):
    history = pd.read_csv(APPROVED_LOG)
    user_history = history[(history['From'] == account) | (history['To'] == account)]
    if user_history.empty:
        return "No transactions found."
    return user_history.to_string(index=False, justify='left')

# View Fraud Logs
def fraud_table():
    try:
        fraud_data = pd.read_csv(FRAUD_LOG)
        return fraud_data.to_string(index=False)
    except Exception as e:
        return f"Error loading fraud log: {e}"

def fraud_plot():
    try:
        fraud_data = pd.read_csv(FRAUD_LOG)
        plt.figure(figsize=(10, 4))
        sns.histplot(fraud_data['Attempted Amount'], bins=20, kde=True, color='red')
        plt.title("Distribution of Fraud Attempt Amounts")
        plt.xlabel("Amount")
        plt.ylabel("Frequency")
        plt.tight_layout()
        plt.savefig("fraud_plot.png")
        return "fraud_plot.png"
    except Exception as e:
        return f"Error generating plot: {e}"

# Banking UI
with gr.Blocks() as ui:
    gr.Markdown("""
    # 🏦 AI-Based Smart Banking System with Fraud Detection
    _Secure transactions, smart banking, live AI protection_
    """)

    with gr.Tab("💸 Transaction"):
        sender = gr.Dropdown(label="Sender", choices=get_accounts())
        receiver = gr.Dropdown(label="Receiver", choices=get_accounts())
        amount = gr.Number(label="Amount")
        submit = gr.Button("Submit")
        result = gr.Textbox(label="Transaction Result")
        s_old = gr.Textbox(label="Sender Old")
        s_new = gr.Textbox(label="Sender New")
        r_old = gr.Textbox(label="Receiver Old")
        r_new = gr.Textbox(label="Receiver New")
        submit.click(handle_transaction, inputs=[sender, receiver, amount], outputs=[result, s_old, s_new, r_old, r_new])

    with gr.Tab("🏦 Deposit"):
        acc = gr.Dropdown(label="Account", choices=get_accounts())
        amt = gr.Number(label="Amount")
        out = gr.Textbox(label="Notification")
        deposit_btn = gr.Button("Deposit")
        deposit_btn.click(deposit, inputs=[acc, amt], outputs=out)

    with gr.Tab("💰 Withdraw"):
        acc2 = gr.Dropdown(label="Account", choices=get_accounts())
        amt2 = gr.Number(label="Amount")
        out2 = gr.Textbox(label="Notification")
        withdraw_btn = gr.Button("Withdraw")
        withdraw_btn.click(withdraw, inputs=[acc2, amt2], outputs=out2)

    with gr.Tab("📊 Balance"):
        acc3 = gr.Dropdown(label="Account", choices=get_accounts())
        out3 = gr.Textbox(label="Balance Info")
        bal_btn = gr.Button("Check Balance")
        bal_btn.click(get_balance, inputs=acc3, outputs=out3)

    with gr.Tab("📄 History"):
        acc4 = gr.Dropdown(label="Account", choices=get_accounts())
        hist_btn = gr.Button("View History")
        hist = gr.Textbox(label="Transaction History", lines=15)
        hist_btn.click(view_history, inputs=acc4, outputs=hist)

    with gr.Tab("🚨 Fraud Analysis"):
        fraud_text = gr.Textbox(label="Fraud Table", lines=10)
        view_table = gr.Button("📋 View Fraud Table")
        #fraud_img = gr.Image(label="Fraud Amount Plot")
        #view_graph = gr.Button("📈 View Fraud Plot")
        view_table.click(fraud_table, outputs=fraud_text)
        #view_graph.click(fraud_plot, outputs=fraud_img)

    with gr.Tab("👤 Account Info"):
        acc_info = gr.Dropdown(label="Account", choices=get_accounts())
        show_info = gr.Button("View Info")
        output_info = gr.Textbox(label="Account Holder Info", lines=6)
        show_info.click(get_account_info, inputs=acc_info, outputs=output_info)


ui.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6ce1afef1eb6acbb8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
print("Model expects features:", model.feature_names_in_)


Model expects features: ['amount' 'sender_balance' 'receiver_balance']


In [ ]:
pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.4 MB/s eta 0:00:00


In [ ]:

import pandas as pd
from faker import Faker
import random

fake = Faker()

# Load your existing accounts.csv
df = pd.read_csv("/content/drive/MyDrive/fraud_detection/accounts.csv")

# Add new columns with fake data
df['Name'] = [fake.name() for _ in range(len(df))]
df['Phone'] = [fake.phone_number() for _ in range(len(df))]
df['Address'] = [fake.address().replace('\n', ', ') for _ in range(len(df))]

# Save the updated file
df.to_csv("/content/drive/MyDrive/fraud_detection/accounts.csv", index=False)

print("✅ accounts.csv updated with Name, Phone, and Address!")


✅ accounts.csv updated with Name, Phone, and Address!
